In [5]:
import requests
import pandas as pd
import json
from pyhocon import ConfigFactory
import os
import time
import numpy as np

In [6]:
conf = ConfigFactory.parse_file('API_Keys.conf')
key = conf.get_string('API.covalent.key')

In [7]:
#wallet parquet file path
pths = r'D:\Teller_Finance\wallet_info_from_covalent\wallet_raw_df.parquet.gzip'

In [13]:
#read all wallet data
df_wallet=pd.read_parquet(pths)

#get address
wallet_address=df_wallet['address'].values

In [15]:
def create_api_string(api, input_dict=None):
    try:
        return api.format(**input_dict)
    except:
        print('Failed to Create Api String')

def secs():
    return np.random.randint(1,5)

In [30]:
class api_data_getter:
    
    def __init__(self,**kwargs):
        self.kwargs = kwargs
        
    
    def covalent_api(self):
        '''
        Returns the sum of two decimal numbers in binary digits.

                Parameters:
                        wallet_address_list (list): list of wallets
                        blockchain_data (str): e.g. balances_v2
                        params (dict) : params={"limit":"100", "key":key } key is necessary
                        number_of_wallets (int) : the number of wallets to loop through default is the full wallet_address_list
                        api_string (str) : api string e.g. "https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"

                Returns:
                        api dataframe output for the requested blockchain_data
        '''
        
        wallet_address = self.kwargs.get('wallet_address_list')
        blockchain_data = self.kwargs.get('blockchain_data') #balances_v2
        params = self.kwargs.get('params') 
        if self.kwargs.get('number_of_wallets'):
            n = self.kwargs.get('number_of_wallets')
        else:
            n= len(wallet_address)# use all wallets
        
        if self.kwargs.get('api_string'):
            api_string = self.kwargs.get('api_string') #params={"limit":"100", "key":key }
        else:
            #some defualt for now
            api_string = "https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"
            
        
        #loop through individual wallets and get data
        df_res= pd.DataFrame()
        for addr in wallet_address[0:n]:
            time.sleep(secs())
            input_dict = {"address":addr, "blockchain_data" :blockchain_data}
            api_= create_api_string(api_string,input_dict)
            r = requests.get(api_ ,params = params)
            print('status of the api request is...')
            print(r)
            print('\n')
            df = pd.read_json(r.content,orient='index').reset_index().rename(columns={'index':'server_output'})
            df = df.loc[ df['server_output'] == 'data'] # get the data component as opposed to error
            df_res = df_res.append(df)
        self.df_res = df_res
        return self
    
    def save_to_parquet(self,file_name):
        self.df_res.to_parquet(file_name + '.gzip' ,compression='gzip')

In [31]:
#example run
params = {"limit":"100", "key":key }
input_dict= {'wallet_address_list':wallet_address,'blockchain_data':'balances_v2','params':params,'number_of_wallets':5,'api_string':"https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"}
adg = api_data_getter(**input_dict)
adg.covalent_api()
adg.save_to_parquet('test_file')

status of the api request is...
<Response [200]>


status of the api request is...
<Response [200]>


status of the api request is...
<Response [200]>


status of the api request is...
<Response [200]>


status of the api request is...
<Response [200]>




In [32]:
f = pd.read_parquet('test_file.gzip')

,server_output,address,chain_id,items,next_update_at,pagination,quote_currency,updated_at
0,data,0xa00b296408521c71a240c58c7909fde265921c45,1.0,"[{'balance': '7120182209304508', 'contract_add...",2021-05-28T05:55:25.432742520Z,NaN,USD,2021-05-28T05:50:25.432742160Z
0,data,0xedfe9173e0a93fd0e4e1653f5d26b0e8548e80db,1.0,"[{'balance': '162535028547844993816', 'contrac...",2021-05-28T05:55:29.845145343Z,NaN,USD,2021-05-28T05:50:29.845145003Z
0,data,0xfa760444a229e78a50ca9b3779f4ce4cce10e170,1.0,"[{'balance': '147700331439528937', 'contract_a...",2021-05-28T05:55:33.240383703Z,NaN,USD,2021-05-28T05:50:33.240383553Z
0,data,0x49d56c2020caae817c3b493b00e5c6dec02f140c,1.0,"[{'balance': '3637885470000000000000', 'contra...",2021-05-28T05:55:34.864951834Z,NaN,USD,2021-05-28T05:50:34.864951335Z
0,data,0x45eb39a39c970ce4278fb8541bf0e3d91527e495,1.0,"[{'balance': '33270985135767694', 'contract_ad...",2021-05-28T05:55:38.253842076Z,NaN,USD,2021-05-28T05:50:38.253841714Z


In [33]:
print(adg.covalent_api.__doc__)


        Returns the sum of two decimal numbers in binary digits.

                Parameters:
                        wallet_address_list (list): list of wallets
                        blockchain_data (str): e.g. balances_v2
                        params (dict) : params={"limit":"100", "key":key } key is necessary
                        number_of_wallets (int) : the number of wallets to loop through default is the full wallet_address_list
                        api_string (str) : api string e.g. "https://api.covalenthq.com/v1/1/address/{address}/{blockchain_data}/"

                Returns:
                        api dataframe output for the requested blockchain_data
        
